In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def Window(wavelength_array, pixel_size, window_size_multiplier, ngal = 1):
        """Returns windows to run the Model function over to speed up calculation
        Parameters: wavelength_array: Full wavelength array of MMT BinoSpec. This is constant
                    ngal: number of galaxies in a given data file
                    pixel_size: width of pixels in wavelength_array
                    window_size_multiplier: Multiple this with pixel size to get width of window in wavelength_array space
        Returns: nwindow_ndarray: l x m x n ndarray where l = ngal, m = number of windows and 
                                n = pixels per windowsize
        """

        nwindow = (wavelength_array[-1] - wavelength_array[0])//pixel_size #number of windows per galaxy. 
                                                        #It is of this form b/c beyond this window exceeds the wavelength_array
        nwindow_array = []

        #Generate nwindow windows
        for i in range(int(nwindow)):
            tmp = np.arange(wavelength_array[i], wavelength_array[i] + pixel_size*(window_size_multiplier), \
                                         pixel_size)
            if(tmp[-1] > (wavelength_array[-1] + pixel_size)):
                break
            else:
                nwindow_array.append(tmp)

        nwindow_array = np.asarray(nwindow_array)
        
        #Repeat nwindow_arary ngal times 
        #https://stackoverflow.com/questions/32171917/copy-2d-array-into-3rd-dimension-n-times-python
        nwindow_ndarray = np.repeat(nwindow_array[np.newaxis, :, :], ngal, axis=0)

        #return nwindow_ndarray
        return nwindow_array

### Generate wavelength grid with each pixel = 2 Angstrom.

In [9]:
pixel_size = .01
wave_grid = np.arange(4500, 4600, pixel_size);
nwindow = (wave_grid[-1] - wave_grid[0])//pixel_size
print(int(nwindow))
print(len(wave_grid))

9999
10000


### Pass the wavelength grid to the Window function to generate windows to run the model over for
### 1 galaxy, and window width = 5 pixels

In [10]:
tmpwindow = Window(wave_grid, ngal = 1, pixel_size = pixel_size, window_size_multiplier=5)
print(tmpwindow.shape)
tmpwindow

(9996, 6)


array([[4500.  , 4500.01, 4500.02, 4500.03, 4500.04, 4500.05],
       [4500.01, 4500.02, 4500.03, 4500.04, 4500.05, 4500.06],
       [4500.02, 4500.03, 4500.04, 4500.05, 4500.06, 4500.07],
       ...,
       [4599.93, 4599.94, 4599.95, 4599.96, 4599.97, 4599.98],
       [4599.94, 4599.95, 4599.96, 4599.97, 4599.98, 4599.99],
       [4599.95, 4599.96, 4599.97, 4599.98, 4599.99, 4600.  ]])

We have generated an m x n ndarray of arrays that consists of m windows and each window contains n pixels

### Model function takes the output of Window function and produces an ndarray to evaluate the model for each window

In [11]:
def ModelO2loop(wavelength_array, width, Amp = 1):
        """Returns the [OII] doublet model
        Parameters: wavelength_array: Full wavelength array of MMT BinoSpec. This is constant and must be odd for lambda0 calc.
                    width: width of the Gaussian doublets. This varies.
                    Amp: Amplitude of the Gaussian doublets
        """
        
        Gaussian = lambda x, mean, std: np.exp(-((x[:, np.newaxis] - mean)/std)**2)

        #Values from http://classic.sdss.org/dr6/algorithms/linestable.html
        separation = (3729.875-3727.092)/2 #separation between lambda0 and the emission lines
        lambda0 = wavelength_array[:,(len(wavelength_array[0])+1)//2] #Take the midpoint of the wavelength_array as the centre
                                                                 #of the Gaussian doublet.wavelength_array must be odd
        
        models = []
        for i,j in enumerate(wavelength_array):
            models.append(Amp*(Gaussian(j, lambda0[i]-separation, width) + \
                               Gaussian(j, lambda0[i]+separation, width)))
            
        models = np.asarray(models)
        
        return models

In [12]:
tmpmodel = ModelO2loop(tmpwindow, width = .5)

In [13]:
tmpmodel

array([[[0.00091117],
        [0.00088585],
        [0.00087075],
        [0.00086573],
        [0.00087075],
        [0.00088585]],

       [[0.00091117],
        [0.00088585],
        [0.00087075],
        [0.00086573],
        [0.00087075],
        [0.00088585]],

       [[0.00091117],
        [0.00088585],
        [0.00087075],
        [0.00086573],
        [0.00087075],
        [0.00088585]],

       ...,

       [[0.00091117],
        [0.00088585],
        [0.00087075],
        [0.00086573],
        [0.00087075],
        [0.00088585]],

       [[0.00091117],
        [0.00088585],
        [0.00087075],
        [0.00086573],
        [0.00087075],
        [0.00088585]],

       [[0.00091117],
        [0.00088585],
        [0.00087075],
        [0.00086573],
        [0.00087075],
        [0.00088585]]])